In [1]:
import json
import os
from library_dicom.dicom_processor.model.Series import Series 
from library_dicom.dicom_processor.model.SeriesPT import SeriesPT
from library_dicom.dicom_processor.model.SeriesCT import SeriesCT
from library_dicom.dicom_processor.model.csv_reader.MaskBuilder import MaskBuilder

In [2]:
json_path = '/media/salim/DD 2To/AHL2011_NIFTI/AHL2011_list_path_csv.json'

In [3]:
data = []
with open(json_path) as json_file : 
    reader = json.load(json_file)
    for info in reader :
        for path in reader[info] :
            data.append(path)

In [4]:
dataset = []
for i in range(0,len(data),3):
    subliste = []
    subliste.append(data[i])
    subliste.append(data[i+1])
    subliste.append(data[i+2])
    dataset.append(subliste)

In [6]:
dataset[11]


['/media/salim/DD 2To/AHL2011_Validated_DICOMS/13011101391019/1.2.124.113532.172.16.4.13.20130416.213156.2050918/1.2.840.113619.2.290.3.279712783.8.1366178187.547/1.2.840.113619.2.290.3.279712783.8.1366178187.547',
 '/media/salim/DD 2To/AHL2011_Validated_DICOMS/13011101391019/1.2.124.113532.172.16.4.13.20130416.213156.2050918/1.2.840.113619.2.290.3.279712783.8.1366178186.947.3/1.2.840.113619.2.290.3.279712783.8.1366178186.947.3',
 'arb_al_apr 17_2013.csv']

In [5]:
#Unconstant spacing 
#dataset[6] 

#['/media/salim/DD 2To/AHL2011_Validated_DICOMS/13011101431012/1.2.124.113532.172.16.25.2.20130121.814.1220583/1.3.46.670589.28.2.12.30.26421.36112.2.3508.0.1359080111/imagys_AHL2011_13011101431012_c72b9b8c-80b3-4803-a240-62a9ec6b6302',
 #'/media/salim/DD 2To/AHL2011_Validated_DICOMS/13011101431012/1.2.124.113532.172.16.25.2.20130121.814.1220583/1.2.840.113704.1.111.3652.1359035909.11/imagys_AHL2011_13011101431012_ce32ad94-1872-4340-9e14-a2b6ec451cf0',
 #'can_sy_jan 24_2013.csv']

#dataset[10]

# ['/media/salim/DD 2To/AHL2011_Validated_DICOMS/13011101391019/1.2.124.113532.172.16.4.13.20130416.213156.2050918/1.2.840.113619.2.290.3.279712783.8.1366178187.547/1.2.840.113619.2.290.3.279712783.8.1366178187.547', '/media/salim/DD 2To/AHL2011_Validated_DICOMS/13011101391019/1.2.124.113532.172.16.4.13.20130416.213156.2050918/1.2.840.113619.2.290.3.279712783.8.1366178186.947.3/1.2.840.113619.2.290.3.279712783.8.1366178186.947.3', 'arb_al_apr 17_2013.csv']

['/media/salim/DD 2To/AHL2011_Validated_DICOMS/13011101331015/1.2.124.113532.10.149.172.5.20131107.93455.7725981/1.2.840.113619.2.290.3.2500492204.268.1383804922.866.3/1.2.840.113619.2.290.3.2500492204.268.1383804922.866.3',
 '/media/salim/DD 2To/AHL2011_Validated_DICOMS/13011101331015/1.2.124.113532.10.149.172.5.20131107.93455.7725981/1.2.840.113619.2.290.3.2500492204.268.1383804923.167/1.2.840.113619.2.290.3.2500492204.268.1383804923.167',
 'con_da_nov 7_2013.csv']

In [5]:
#index des unconstant sapcing serie = 6, 11
dataset.pop(6)
dataset.pop(11)


['/media/salim/DD 2To/AHL2011_Validated_DICOMS/13011101391021/1.2.124.113532.172.16.4.13.20130618.214008.2108115/1.2.840.113619.2.290.3.279712783.738.1371622390.965/1.2.840.113619.2.290.3.279712783.738.1371622390.965',
 '/media/salim/DD 2To/AHL2011_Validated_DICOMS/13011101391021/1.2.124.113532.172.16.4.13.20130618.214008.2108115/1.2.840.113619.2.290.3.279712783.738.1371622390.664.3/1.2.840.113619.2.290.3.279712783.738.1371622390.664.3',
 'lop_jo_jun 19_2013.csv']

In [8]:
len(dataset)

615

In [7]:
#serie1 = dataset[0]
#print(serie1)
nifti_directory = '/media/salim/DD 2To/AHL2011_NIFTI'


csv_directory = '/media/salim/DD 2To/AHL2011_CSV_DATA/AHL2011-CSV'

cpt = 0
for serie in dataset : 
    print(serie)
#serie CT / PT 
    for i in range(2):
        #print(i)
        #recupérer study IUD et CT PT dans le tri des données, dans le json 
        serie_object = Series(serie[i])
        serie_data = serie_object.get_series_details()
        modality = serie_data['series']['Modality'] #CT ou PT
        print(modality)
        study_uid = serie_data['study']['StudyInstanceUID']
        filename = study_uid+'_'+'nifti_'+modality+'.nii'


        if modality == 'CT' :
            serie_ct_objet = SeriesCT(serie[i])
            serie_ct_objet.get_numpy_array()
            serie_ct_objet.export_nifti(os.path.join(nifti_directory, filename))
            

        if modality == 'PT':
            print('serie PT, generation du mask avec fichier csv associée')
            #print('1')
            serie_pt_objet = SeriesPT(serie[i])
            #print('2')
            nifti_array = serie_pt_objet.get_numpy_array()
            #print('3')

            size_matrix = serie_pt_objet.get_size_matrix()
            print(size_matrix)

            mask_objet = MaskBuilder(os.path.join(csv_directory, serie[2]), size_matrix)
            mask_4D = mask_objet.mask_array
            print(mask_objet.calcul_suv(serie_pt_objet.get_numpy_array()))
            if mask_objet.is_correct_suv(serie_object.get_numpy_array()) == False : 
                print("erreur calcul suv, philips")
                print(mask_objet.ecart_suv_max(nifti_array))

                

                
                
            
            serie_object.export_nifti(os.path.join(nifti_directory, filename))

    cpt += 1


['/media/salim/DD 2To/AHL2011_Validated_DICOMS/13011101331015/1.2.124.113532.10.149.172.5.20131107.93455.7725981/1.2.840.113619.2.290.3.2500492204.268.1383804922.866.3/1.2.840.113619.2.290.3.2500492204.268.1383804922.866.3', '/media/salim/DD 2To/AHL2011_Validated_DICOMS/13011101331015/1.2.124.113532.10.149.172.5.20131107.93455.7725981/1.2.840.113619.2.290.3.2500492204.268.1383804923.167/1.2.840.113619.2.290.3.2500492204.268.1383804923.167', 'con_da_nov 7_2013.csv']
CT
PT
serie PT, generation du mask avec fichier csv associée
[256, 256, 299]
{1: {'SUV_max': 4.52, 'SUV_mean': 2.66, 'SD': 0.55}, 2: {'SUV_max': 8.19, 'SUV_mean': 4.58, 'SD': 1.14}, 3: {'SUV_max': 3.58, 'SUV_mean': 2.16, 'SD': 0.53}, 4: {'SUV_max': 8.34, 'SUV_mean': 5.24, 'SD': 1.28}, 5: {'SUV_max': 9.27, 'SUV_mean': 5.31, 'SD': 1.14}, 6: {'SUV_max': 5.78, 'SUV_mean': 3.27, 'SD': 0.63}, 7: {'SUV_max': 11.71, 'SUV_mean': 6.21, 'SD': 1.23}, 8: {'SUV_max': 13.11, 'SUV_mean': 7.16, 'SD': 1.38}, 9: {'SUV_max': 12.21, 'SUV_mean': 

Exception: Unconstant Spacing

In [61]:
None

In [8]:
list_manufacturer = []
for serie in dataset : 
#serie CT / PT 
    for i in range(2):
        #print(i)
        #recupérer study IUD et CT PT dans le tri des données, dans le json 
        serie_object = Series(serie[i])
        serie_data = serie_object.get_series_details()
        modality = serie_data['series']['Modality'] #CT ou PT

        if modality == 'PT' :
            serie_pt = SeriesPT(serie[i])
            list_manufacturer.append(serie_pt.get_series_details()['series']['Manufacturer'])

print(list_manufacturer)

['GE MEDICAL SYSTEMS', 'Philips Medical Systems', 'Philips Medical Systems', 'Philips Medical Systems', 'Philips Medical Systems', 'Philips Medical Systems', 'Philips Medical Systems', 'Philips Medical Systems', 'Philips Medical Systems', 'Philips Medical Systems', 'GE MEDICAL SYSTEMS', 'SIEMENS', 'SIEMENS', 'SIEMENS', 'SIEMENS', 'SIEMENS', 'SIEMENS', 'SIEMENS', 'SIEMENS', 'SIEMENS', 'SIEMENS', 'SIEMENS', 'SIEMENS', 'SIEMENS', 'SIEMENS', 'SIEMENS', 'SIEMENS', 'SIEMENS', 'SIEMENS', 'SIEMENS', 'SIEMENS', 'SIEMENS', 'SIEMENS', 'GE MEDICAL SYSTEMS', 'GE MEDICAL SYSTEMS', 'GE MEDICAL SYSTEMS', 'GE MEDICAL SYSTEMS', 'GE MEDICAL SYSTEMS', 'GE MEDICAL SYSTEMS', 'CPS', 'CPS', 'CPS', 'CPS', 'CPS', 'CPS', 'CPS', 'CPS', 'CPS', 'CPS', 'CPS', 'CPS', 'CPS', 'Philips Medical Systems', 'Philips Medical Systems', 'Philips Medical Systems', 'Philips Medical Systems', 'Philips Medical Systems', 'Philips Medical Systems', 'Philips Medical Systems', 'Philips Medical Systems', 'Philips Medical Systems', 'Phi

In [22]:
serie_one = data[0::3] 
serie_two = data[1::3]
number_serie = len(serie_one)
all_serie = []
for i in range(number_serie):
    serie = []
    serie.append(serie_one[i])
    serie.append(serie_two[i])
    all_serie.append(serie)


In [39]:
None

In [40]:
None

In [32]:
None